Programming in Toki Pona
============

Firstly, an FAQ:

What is toki pona?
- 
toki pona (always written in lowecase) is a conlang (constructed language) created by Sonja Lang, and is known for its small vocabulary and general simplicity. It has around 130 words, no conjugations/declensions or gender and every word can be either a noun, a verb or an adjective depending on context. Its numbering system is also super simple, similar to the roman numerals.

It was chosen to be used in this programming language because i think its a fun challenge to translate 'harder' concepts such as multiplication and if/elses into such a simple language, and also its appearence when using the logography. By the way, this is how you say 'toki pona' in toki pona:

![toki pona logo](./assets/smol.png)

How does it look like?
- 
There are two ways to view this language, the first one is using the romanization, and the second one is using toki pona's logography. Both can be seen below:

![Sample code in both views](./assets/sampleCode_compare.png "Sample code in both views")

The romanization is the one that will appear here in the code snippets and such. If you want to see it in its logographical form, there is a simple way to see it in an html, as we can see:

Its done by pulling an external toki pona style sheet into the html, as so:

``` 
<head>
    <link rel="stylesheet" href="https://davidar.github.io/linja-pona/stylesheet.css"></link>
</head>
<body class="linja-pona" style="font-size: xx-large">
    toki+pona li pona tawa mi 
</body>
```

This exact sample can be [easily seen here](./logography/firstExample.html). Another one, as seen above, looks [like this](./logography/secondExample.html).
I cant seem to make this link open in the browser, but you should be able to open it ([maybe even inside your IDE](vscode:extension/george-alisson.html-preview-vscode), with some extensions). The only differences in the html are the ```<br>``` used for line breaks and ```&emsp;``` used for spaces (not necessary, but more read-able).

The brackets in the variable names are used simply to allow the correct visualization in logography, for the [cartuches](https://en.wikipedia.org/wiki/Cartouche "Variable names like: [_nanpa_wan]").

What commands are in it?
- 
Currently, there are:

- 3 variable types, 'nimi' for strings, 'nanpa' for integers and 'kulupu' for an array
    - all variables are always global
- variable names in cartuches
- number assignment using the toki pona numbering system
- operations such as: sum, subtraction, multiplication, division
- if-else statements with 'la (condition) ni (commands) ante la (commands) pini'
    - note that only the 'equal to' comparison exists (you have to get creative with the else!)
- while statements with 'awen la (condition) ni (commands) pini'
    - same comparison restriction 
- 'toki' for printing out expressions 
    - allows for any type of value, be it a variable, string or number
- 'kute' for inputs
- 'pali' for functions
    - they dont take 'arguments' per se, but can work similar to assembly function calling


What now?
- 

Now we can go into the coding portion of this notebook! :D

Feel free to skip to the end though, as there is another fun thing you can do...

----------------------------------------
----------------------------------------

```
<program> ::= <declarations><commands>

<declarations> ::=  <declaration> |
                   <declaration> <declarations> |
                   <function> |
                   <function> <declarations>

<declaration> ::= nimi id |
                 nanpa id |
                 kulupu id 

<function> ::= pali id ni <commands> kama <expression> pini

<commands> ::=  <atribution> |
                <atribution> <commands> |
                <if-else> |
                <if-else> <commands> |
                <while> |
                <while> <commands> |
                <print> |
                <print> <commands> |

<atribution> ::= id sama <expression>

<expression> ::= id |
                 pali id |
                 <compounds> |
                 <string> |
                 <expression> en <expression> |
                 <expression> ala en <expression> |
                 <expression> namako <expression> |
                 <expression> kipisi <expression> |

<compounds> ::= <simple> |
                <simple> ala |
                <simple> <compounds> 
                <simple> <compounds> ala

<simple> ::= wan |
            tu |
            luka |
            mute |
            ale

<print> ::= toki <expression>

<input> ::= kute id

<if-else> ::= la (<expression> sama sama <expression>) ni <commands> ante la <commands> pini |
              la (<expression> sama sama <expression>) ante la <commands> pini

<while> ::= awen la (<expression> sama sama <expression>) ni <commands> pini |

```

------------------------------
------------------------------

Below, is where the code starts! Its all commented so you should be able to follow along:

In [11]:
# Sample code - will be used for testing purposes
# make sure to 'pip install rply' if you haven't already!

# ------------------------------------------------ #

# '''
# nanpa [_nanpa_wan] 
# nanpa [_nanpa_tu] 
# nanpa [_nanpa_tu_wan] 

# [_nanpa_wan] sama luka wan
# [_nanpa_tu] sama tu

# [_nanpa_tu_wan] sama [_nanpa_wan] en [_nanpa_tu]
# '''

# ------------------------------------------------ #

# [_nanpa_tu_wan] should be equal to 8 (luka tu wan) btw
# code will be in a string at the last step


In [12]:
# LexerGenerator - transforms all of the code inputted into tokens
from rply import LexerGenerator

lg = LexerGenerator()

# This part of the code transforms all of the code we will input into tokens
# The tokens will then later be used to parse the code into a tree

lg.add('ID', r'\[(\_([jklmnpstw]?[aeiou][n]?)+)+\]') 		# how does this work this perfectly
lg.add('WORDS', r'\"[a-zA-Z_ \\\(\)\?\!\:]+\"')   # no limitations for the letters in strings here, unlike the variable names

lg.add('INT', r'nanpa')
lg.add('STR', r'nimi')
lg.add('ARR', r'kulupu')

lg.add('FUNCT', r'pali')
lg.add('RETURN', r'kama')

lg.add('WHILE', r'awen la')
lg.add('ELSE', r'ante la')
lg.add('IF', r'la')
lg.add('THEN', r'ni')
lg.add('END', r'pini')
lg.add('COMP', r'sama sama')

lg.add('PRINT', r'toki')
lg.add('INPUT', r'kute')

lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('EQUALS', r'sama')
lg.add('SUB', r'ala en')
lg.add('ADD', r'en')
lg.add('MUL', r'namako')
lg.add('DIV', r'kipisi')

lg.add('SIMPLE', r"\b(?:ale|mute|luka|tu|wan)\b")	# it will separate all of the numbers, this will be dealt with shortly
lg.add('NEG', r"ala")	# this is a negative number

lg.ignore('\s+')    # ignores all whitespace

lexer = lg.build()

In [13]:
# BaseBox - Preparation for the next step
from rply.token import BaseBox

class Program(BaseBox):
    def __init__(self, decls,cmmds):
        self.decls = decls
        self.cmmds = cmmds

    def accept(self, visitor):
        visitor.visit_program(self)

# ------------------------------------------- #
# Declarations

class Declarations(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_declarations(self)

class Declaration(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp

    def accept(self, visitor):
        visitor.visit_declaration(self)

# ------------------------------------------- #
# Commands

class Commands(BaseBox):
    def __init__(self, cmmd,cmmds):
        self.cmmd = cmmd
        self.cmmds = cmmds

    def accept(self, visitor):
        return visitor.visit_commands(self)

class Attribution(BaseBox):
    def __init__(self, id, expr, index):
        self.id = id
        self.expr = expr
        self.index = index

    def accept(self, visitor):
        return visitor.visit_attribution(self)

# ------------------------------------------- #

class Function(BaseBox):
    def __init__(self, id, expr, ret):
        self.id = id
        self.expr = expr
        self.ret = ret

    def accept(self, visitor):
        return visitor.visit_function(self)

class CallFunction(BaseBox):
    def __init__(self, id):
        self.id = id

    def accept(self, visitor):
        return visitor.visit_callfunction(self)

# ------------------------------------------- #

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        return visit(self)

class IfElse(BaseBox):
    def __init__(self, expr1, expr2, cmmdif, cmmdelse):
        self.expr1=expr1
        self.expr2=expr2
        self.cmmdif=cmmdif
        self.cmmdelse=cmmdelse

    def accept(self, visitor):
        visitor.visit_ifelse(self)

class While (BaseBox):
    def __init__(self, expr1, expr2, cmmds):
        self.expr1=expr1
        self.expr2=expr2
        self.cmmds=cmmds

    def accept(self, visitor):
        visitor.visit_while(self)

# ------------------------------------------- #
# Number and its classifications

class Compounds(Expr):
    def __init__(self, simple, compounds,neg):
        self.simple = simple
        self.compounds = compounds
        self.neg = neg
    def accept(self, visitor):
        return visitor.visit_compounds(self)

# ------------------------------------------- #
# Other expressions

class Print(Expr):
    def __init__(self, value):
        self.value = value
    def accept(self, visitor):
        visitor.visit_print(self)

class Input(Expr):
    def __init__(self, id):
        self.id = id
    def accept(self, visitor):
        visitor.visit_input(self)

class Id(Expr):
    def __init__(self, value, index):
        self.value = value
        self.index = index
    def accept(self, visitor):
        return visitor.visit_id(self)
    
class String(Expr):
    def __init__(self, value):
        self.value = value
    def accept(self, visitor):
        return visitor.visit_string(self)

# -- the basic binary operations, they are all handeled by the BinaryOp class -- #
class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right
class Add(BinaryOp):
  pass
class Sub(BinaryOp):
  pass
class Mul(BinaryOp):
  pass
class Div(BinaryOp):
  pass


In [14]:
# ParserGenerator - Transforms the token list into a tree
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['EQUALS', 'COMP',
     'ADD', 'SUB', 'MUL', 'DIV', 
     'WHILE', 'IF', 'ELSE', 'THEN', 'END', 
     'ID', 'SIMPLE', 'NEG',
     'WORDS',
     'INT', 'STR', 'ARR', 'FUNCT', 'RETURN',
     'OPEN_PARENS', 'CLOSE_PARENS', 
     'PRINT', 'INPUT'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['ADD', 'SUB']),
        ('left', ['MUL', 'DIV'])
    ]
)

# ------------------------------------------- #

@pg.production('program : declarations commands')
def program(p):
    return Program(p[0],p[1])

@pg.production('declarations : declaration')
def declarations(p):
    return Declarations(p[0],None)
@pg.production('declarations : declaration declarations')
def declarations(p):
    return Declarations(p[0],p[1])

@pg.production('declarations : function')
def declarations(p):
    return Declarations(p[0],None)
@pg.production('declarations : function declarations')
def declarations(p):
    return Declarations(p[0],p[1])

@pg.production('declaration : INT ID')
def declaration_integer(p):
    return Declaration(p[1].getstr(), "int")

@pg.production('declaration : STR ID')
def declaration_integer(p):
    return Declaration(p[1].getstr(), "str")

@pg.production('declaration : ARR ID')
def declaration_integer(p):
    return Declaration(p[1].getstr(), "arr")

@pg.production('function : FUNCT ID THEN commands RETURN expression END')
def declaration_function(p):
    return Function(p[1].getstr(), p[3], p[5])

# ------------------------------------------- #
# Commands (anything that isnt a declaration)

@pg.production('commands : attribution commands')
def command_commands(p):
    return Commands(p[0],p[1])
@pg.production('commands : attribution')
def commands_command(p):
    return Commands(p[0],None)

@pg.production('commands : if-else commands')
def command_commands(p):
    return Commands(p[0],p[1])
@pg.production('commands : if-else')
def command_commands(p):
    return Commands(p[0],None)

@pg.production('commands : while commands')
def command_commands(p):
    return Commands(p[0],p[1])
@pg.production('commands : while')
def command_commands(p):
    return Commands(p[0],None)

@pg.production('commands : print commands')
def command_commands(p):
    return Commands(p[0],p[1])
@pg.production('commands : print')
def command_commands(p):
    return Commands(p[0],None)

@pg.production('commands : input commands')
def command_commands(p):
    return Commands(p[0],p[1])
@pg.production('commands : input')
def command_commands(p):
    return Commands(p[0],None)

# ------------------------------------------- #
# Commands

@pg.production('attribution : ID EQUALS expression')
def commands_attribution(p):
    return Attribution(p[0].getstr(), p[2], None)

@pg.production('attribution : ID OPEN_PARENS compounds CLOSE_PARENS EQUALS expression')
def commands_attribution(p):
    return Attribution(p[0].getstr(), p[5], p[2])

@pg.production('if-else : IF OPEN_PARENS expression COMP expression CLOSE_PARENS THEN commands ELSE commands END')
def expression_ifelse1(p):
    return IfElse (p[2],p[4],p[7],p[9])
@pg.production('if-else : IF OPEN_PARENS expression COMP expression CLOSE_PARENS THEN commands END')
def expression_ifelse2(p):
    return IfElse (p[2],p[4],p[7],None)
@pg.production('if-else : IF OPEN_PARENS expression COMP expression CLOSE_PARENS ELSE commands END ')
def expression_ifelse3(p):
    return IfElse (p[2],p[4],None,p[7])

@pg.production('while : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS THEN commands END')
def expression_while(p):
    return While (p[2],p[4],p[7])

@pg.production('print : PRINT expression')
def expression_print(p):
    return Print(p[1])

@pg.production('input : INPUT ID')
def expression_input(p):
    return Input(p[1].getstr())

# ------------------------------------------- #

@pg.production('compounds : SIMPLE')
def compounds_compound(p):
    return Compounds(p[0],None, 1)
@pg.production('compounds : SIMPLE NEG')
def compounds_compound2(p):
    return Compounds(p[0],None, -1)
@pg.production('compounds : SIMPLE compounds')
def compound_compounds3(p):
    return Compounds(p[0],p[1], 1)
@pg.production('compounds : SIMPLE compounds NEG')
def compound_compounds4(p):
    return Compounds(p[0],p[1], -1)

# ------------------------------------------- #

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr(), None)
@pg.production('expression : ID OPEN_PARENS compounds CLOSE_PARENS')
def expression_id(p):
    return Id(p[0].getstr(), p[2])

@pg.production('expression : WORDS')
def expression_id(p):
    return String(p[0].getstr())

@pg.production('expression : FUNCT ID')
def expression_function(p):
    return CallFunction(p[1].getstr())

@pg.production('expression : SIMPLE')
def compounds_compound(p):
    return Compounds(p[0],None, 1)
@pg.production('expression : SIMPLE NEG')
def compounds_compound2(p):
    return Compounds(p[0],None, -1)
@pg.production('expression : SIMPLE compounds')
def compound_compounds3(p):
    return Compounds(p[0],p[1], 1)
@pg.production('expression : SIMPLE compounds NEG')
def compound_compounds4(p):
    return Compounds(p[0],p[1], -1)

@pg.production('expression : expression ADD expression')
@pg.production('expression : expression SUB expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'ADD':
        return Add(left, right)
    elif p[1].gettokentype() == 'SUB':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

@pg.error
def error_handler(token):
    raise ValueError("Ran into a %s where it wasn't expected" % token.gettokentype())

parser = pg.build()

C:\Users\julia\AppData\Local\Temp\ipykernel_19940\2372206508.py:197: ParserGeneratorWarning: 2 shift/reduce conflicts
  parser = pg.build()


In [15]:
# Symbol Table - Creates a list of all of the variables and their types
ST={}
# Stores the functions
functions={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_program(self, prog):
        prog.decls.accept(self)
        prog.cmmds.accept(self)

    def visit_declarations(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_declaration(self, d):
        ST[d.id]=d.tp

    def visit_function(self, d):
        ST[d.id]='int'
        functions[d.id]=d
        
    def visit_attribution(self, d):
      pass
    def visit_commands(self, d):
      pass


In [16]:
# Decorator - Goes through the tree and 'decorates' it with the types of the variables
class Decorator(Visitor):

    def visit_program(self, i):
        i.cmmds.accept(self)

    # ------------------------------------------- #

    def visit_commands(self, d):
        d.cmmd.accept(self)
        if d.cmmds!=None:
          d.cmmds.accept(self)

    def visit_attribution(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)

    # ------------------------------------------- #

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.cmmds.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        if i.cmmdif!=None:
          i.cmmdif.accept(self)
        if i.cmmdelse!=None:
          i.cmmdelse.accept(self)
    # ------------------------------------------- #
    
    def visit_print(self, i):
        pass
    
    def visit_input(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          print(ST, i.id)
          raise AssertionError('id not declared')
    
    def visit_string(self, i):
        i.decor_type="str"

    def visit_compounds(self, i):
        if i.compounds!=None:
          i.compounds.accept(self)
        i.decor_type="int"

    # ------------------------------------------- #

    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id not declared')
        
    def visit_callfunction(self, i):
        if i.id in ST:
          i.decor_type=ST[i.id]
        else:
          raise AssertionError('function not declared')

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        elif a.left.decor_type=="str" and a.right.decor_type=="str":
          a.decor_type="str"
        elif a.left.decor_type=="arr" or a.right.decor_type=="arr":
          pass # not an issue!
        else:
          raise AssertionError('id values incompatible')

    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        elif a.left.decor_type=="str" and a.right.decor_type=="str":
          a.decor_type="str"
        elif a.left.decor_type=="arr" or a.right.decor_type=="arr":
          pass # not an issue!
        else:
          raise AssertionError('id values incompatible')

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        elif a.left.decor_type=="str" and a.right.decor_type=="str":
          a.decor_type="str"
        elif a.left.decor_type=="arr" or a.right.decor_type=="arr":
          pass # not an issue!
        else:
          raise AssertionError('id values incompatible')

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
        elif a.left.decor_type=="str" and a.right.decor_type=="str":
          a.decor_type="str"
        elif a.left.decor_type=="arr" or a.right.decor_type=="arr":
          pass # not an issue!
        else:
          raise AssertionError('id values incompatible')


In [17]:
# TypeVerifier - Checks if all of the types in variable assignments are correct
class TypeVerifier(Visitor):

    def visit_program(self, i):
        i.cmmds.accept(self)

    def visit_commands(self, d):
        d.cmmd.accept(self)
        if d.cmmds!=None:
          d.cmmds.accept(self)

    def visit_print(self, i):
        pass

    def visit_input(self, i):
        pass

    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')
    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')


    def visit_attribution(self, i):
        if i.decor_type=="int" and i.expr.decor_type!="int":
            raise AssertionError('type error')
        if i.decor_type=="str" and i.expr.decor_type!="str":
            raise AssertionError('type error')
        # array can recieve both, no verification needed
        

In [18]:
# Eval - Runs the code
variables={}
functions={}
class Eval(Visitor):

  def __init__(self):
    self.test=0  
  
  def visit_program(self,var):
    var.cmmds.accept(self)

  def visit_commands(self,var):
    var.cmmd.accept(self)
    if (var.cmmds!=None):
      var.cmmds.accept(self)

  def visit_command(self,var):
    var.cmmd.accept(self)

  def visit_attribution(self,var):
    result=var.expr.accept(self)
    if (var.index!=None):
      index=var.index.accept(self)
      if var.id not in variables:
        variables[var.id]={}
      variables[var.id][index]=result
    else:
      variables[var.id]=result

  # ----------------------------------------------------- #

  def visit_ifelse(self, i):
    if(i.expr1.accept(self) == i.expr2.accept(self)):
      if i.cmmdif!=None:
        i.cmmdif.accept(self)
    else:
      if i.cmmdelse!=None:
        i.cmmdelse.accept(self)

  def visit_while(self, i):
    while (i.expr1.accept(self) == i.expr2.accept(self)):
        i.cmmds.accept(self)

  # ----------------------------------------------------- #
  # Other important functions
  
  def visit_id(self,i):
    if i.value not in variables:
      print(variables, i.value)
      raise AssertionError('id uninitialized')
    if (i.index!=None):           # for arrays
      index = i.index.accept(self)
      return variables[i.value][index]
    return variables[i.value]
  
  def visit_string(self,var):
    value = var.value[1:-1]
    return value
  
  def visit_callfunction(self,var):
    if var.id not in functions:
      raise AssertionError('function uninitialized')
    func = functions[var.id]
    func.expr.accept(self)
    return func.ret.accept(self)
  
  def visit_compounds(self,number):
    value = str(number.simple)
    ones = value.count('wan')
    twos = value.count('tu')
    fives = value.count('luka')
    twentys = value.count('mute')
    hundreds = value.count('ale')
    trueNumber = ones + twos*2 + fives*5 + twentys*20 + hundreds*100
    if (number.compounds!=None):
      trueNumber += number.compounds.accept(self)
    return trueNumber*number.neg

  def visit_print(self,var):
    value = var.value.accept(self)
    if type(value) != int and type(value) != dict:
      value = value.replace("\\n", "\n") # due to problems with encoding
    print(value, end=" ")

  def visit_input(self,i):
    inp = input()
    if ST[i.id] == "int":
      if inp.isnumeric() == False:
          raise AssertionError('String in int variable')
      inp = int(inp)
    variables[i.id] = inp

  # ----------------------------------------------------- #
  # arithmetic operations

  def visit_add(self,add):
    return add.left.accept(self)+add.right.accept(self)
  def visit_sub(self,add):
    return add.left.accept(self)-add.right.accept(self)
  def visit_mul(self,add):
    return add.left.accept(self)*add.right.accept(self)
  def visit_div(self,add):
    return add.left.accept(self)/add.right.accept(self)

Code done! now for the

----------------------

Testing
---------------------

In [19]:
# All the small tests, in one cell as to not clutter the notebook
# Their outputs are cleaner to read by the way

# ------------------------------------------------------------------------------- #
# Simple first test (adds up 6 and 2, then prints it)
'''
nanpa [_nanpa_wan] 
nanpa [_nanpa_tu] 
nanpa [_nanpa_tu_wan] 

[_nanpa_wan] sama luka wan
[_nanpa_tu] sama tu

[_nanpa_tu_wan] sama [_nanpa_wan] en [_nanpa_tu]
toki [_nanpa_tu_wan]
'''

# program above in a string
simpleTesting = "nanpa [_nanpa_wan] nanpa [_nanpa_tu] nanpa [_nanpa_tu_wan] [_nanpa_wan] sama luka wan [_nanpa_tu] sama tu [_nanpa_tu_wan] sama [_nanpa_wan] en [_nanpa_tu] toki [_nanpa_tu_wan]"

print("Test: ", "simple sum", "\n")

# 'compiles' the code
lexy=lexer.lex(simpleTesting)
arvore=parser.parse(lexy)
ST={} # resets the symbol table
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())

# runs the program
variables={} # resets the variables
functions={} # resets the functions
arvore.accept(Eval())

# prints the variables
print("\n\nVariables: ", variables)

# ------------------------------------------------------------------------------- #
# Testing arrays (adds up arr[1] and arr[2] and saves in arr[3], then prints it)
'''
kulupu [_kulupu] 
nanpa [_nanpa] 

[_kulupu] (wan) sama luka wan 
[_nanpa] sama tu 

[_kulupu] (wan) sama [_nanpa] en [_kulupu] (wan) 
toki [_kulupu]
'''

testArray = "kulupu [_kulupu] [_kulupu] (wan) sama luka wan [_kulupu] (tu) sama tu [_kulupu] (wan tu) sama [_kulupu] (tu) en [_kulupu] (wan) toki [_kulupu]"
print("---------------------------------------------")
print("\nTest: ", "arrays", "\n")

variables={} # resets the variables
functions={} # resets the functions
ST={} # resets the symbol table

lexy=lexer.lex(testArray)
arvore=parser.parse(lexy)
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(Eval())
print("\n\nVariables: ", variables)

# ------------------------------------------------------------------------------- #
# Testing strings (adds up "kulupu" and "nanpa" and saves in "kulupu nanpa", then prints it)
'''
nimi [_kulupu] 
nimi [_nanpa] 
nimi [_kulupu_nanpa] 

[_kulupu] sama "kulupu" 
[_nanpa] sama "nanpa"

[_kulupu_nanpa] sama [_kulupu] en " " en [_nanpa]
toki [_kulupu_nanpa]
'''

testString = "nimi [_kulupu] nimi [_nanpa] nimi [_kulupu_nanpa] [_kulupu] sama \"kulupu\" [_nanpa] sama \"nanpa\" [_kulupu_nanpa] sama [_kulupu] en \" \" en [_nanpa] toki [_kulupu_nanpa]"
print("---------------------------------------------")
print("\nTest: ", "strings", "\n")

variables={} # resets the variables
functions={} # resets the functions
ST={} # resets the symbol table

lexy=lexer.lex(testString)
arvore=parser.parse(lexy)
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(Eval())
print("\n\nVariables: ", variables)
# ------------------------------------------------------------------------------- #
# Testing ifs and whiles (loop that prints 1 to 9)
'''
nanpa [_lete]
nanpa [_luka_sike]

[_lete] sama wan
[_luka_sike] sama wan

awen la ([_lete] sama sama wan) ni
    la ([_luka_sike] sama sama luka luka) ni
        [_lete] sama tu
    ante la
        toki [_luka_sike]
        [_luka_sike] sama [_luka_sike] en wan
    pini
pini
'''

testIfsWhiles = "nanpa [_lete] nanpa [_luka_sike] [_lete] sama wan [_luka_sike] sama wan awen la ([_lete] sama sama wan) ni la ([_luka_sike] sama sama luka luka) ni [_lete] sama tu ante la toki [_luka_sike] [_luka_sike] sama [_luka_sike] en wan pini pini"
print("---------------------------------------------")
print("\nTest: ", "ifs and whiles", "\n")

variables={} # resets the variables
functions={} # resets the functions
ST={} # resets the symbol table

lexy=lexer.lex(testIfsWhiles)
arvore=parser.parse(lexy)
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(Eval())
print("\n\nVariables: ", variables)

# ------------------------------------------------------------------------------- #
# Testing input and output (input a number and a string, then print them)
'''
nanpa [_nanpa]
nimi [_sitelen]

toki "input a number"
kute [_nanpa]
toki "\n"
toki [_nanpa]

toki "\ninput a string"
kute [_sitelen]
toki "\n"
toki [_sitelen]
'''

testInput = "nanpa [_nanpa] nimi [_sitelen] toki \"input a number\" kute [_nanpa] toki \"\\n\" toki [_nanpa] toki \"\\ninput a string\" kute [_sitelen] toki \"\\n\" toki [_sitelen]"
print("---------------------------------------------")
print("\nTest: ", "inputs", "\n")

variables={} # resets the variables
functions={} # resets the functions
ST={} # resets the symbol table

lexy=lexer.lex(testInput)
arvore=parser.parse(lexy)
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(Eval())
print("\n\nVariables: ", variables)

# ------------------------------------------------------------------------------- #
# Testing function calls (calls a function that adds 1 to a number)
'''
nanpa [_nanpa] 

pali [_ma_en_wan] ni 
    [_nanpa] sama [_nanpa] en wan
    toki [_nanpa]
    kama [_nanpa] 
pini

[_nanpa] sama wan
[_nanpa] sama pali [_ma_en_wan]
'''

testArray = "nanpa [_nanpa] \npali [_ma_en_wan] ni \n[_nanpa] sama [_nanpa] en wan \ntoki [_nanpa] \nkama [_nanpa] \npini \n[_nanpa] sama wan \n[_nanpa] sama pali [_ma_en_wan]"
print("---------------------------------------------")
print("\nTest: ", "Functions", "\n")

variables={} # resets the variables
functions={} # resets the functions
ST={} # resets the symbol table

lexy=lexer.lex(testArray)
arvore=parser.parse(lexy)
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(Eval())
print("\n\nVariables: ", variables)


Test:  simple sum 

8 

Variables:  {'[_nanpa_wan]': 6, '[_nanpa_tu]': 2, '[_nanpa_tu_wan]': 8}
---------------------------------------------

Test:  arrays 

{1: 6, 2: 2, 3: 8} 

Variables:  {'[_kulupu]': {1: 6, 2: 2, 3: 8}}
---------------------------------------------

Test:  strings 

kulupu nanpa 

Variables:  {'[_kulupu]': 'kulupu', '[_nanpa]': 'nanpa', '[_kulupu_nanpa]': 'kulupu nanpa'}
---------------------------------------------

Test:  ifs and whiles 

1 2 3 4 5 6 7 8 9 

Variables:  {'[_lete]': 2, '[_luka_sike]': 10}
---------------------------------------------

Test:  inputs 

input a number 


 12 
input a string 
 ab 

Variables:  {'[_nanpa]': 12, '[_sitelen]': 'ab'}
---------------------------------------------

Test:  Functions 

2 

Variables:  {'[_nanpa]': 2}


------------------------------
------------------------------

Ok, so now its over?
-

Actually, no! There is another file called 'codeRunner.py' in the root of all of this, if you run it, you will be able to see a more interactive code than this one, with commands and an ability to run custom toki pona scripts. Feel free to explore it and find out more for yourself though!